# Exploring spatial filtering, high pass
Joshua Stough
DIP 3.6 on high-pass. See [correlate](https://docs.scipy.org/doc/scipy/reference/generated/scipy.ndimage.correlate.html)

Here we're going to look at the high-pass spatial filter, which derives from the local second derivative. See [these cool visualizations](http://campar.in.tum.de/Chair/HaukeHeibelGaussianDerivatives) of the Gaussian and its derivatives, and [read here](https://homepages.inf.ed.ac.uk/rbf/HIPR2/log.htm).

In [ ]:
%matplotlib widget
import matplotlib.pyplot as plt
import scipy.ndimage as ndimage
import numpy as np

from scipy.ndimage import correlate
from skimage.filters import *

plt.style.use('dark_background')

In [ ]:
def arrInfo(I):
    return I.shape, I.min(), I.max(), I.dtype

In [ ]:
I = plt.imread('cat_small.png').astype('float')
arrInfo(I)

In [ ]:
plt.figure()
plt.imshow(I)

In [ ]:
# The image includes an alpha channel that we're not going to need.
I = I[...,:3].copy()
arrInfo(I)

&nbsp;

## High-pass filtering, involves computing a pixel's difference from its neighborhood

In [ ]:
laplace_h = -1*np.ones((3,3))
laplace_h[1,1] = 8
laplace_h

In [ ]:
I_laplace = np.stack([correlate(I[...,i], laplace_h) for i in range(3)], axis=-1)

In [ ]:
f, ax = plt.subplots(1,2, figsize=(10,5), sharex=True, sharey=True)
ax[0].imshow(I)
ax[0].set_title('Original')
ax[1].imshow(I_laplace, cmap='gray')
ax[1].set_title('LaPlace Filter')

&nbsp;

## Some fraction of the LaPlace can simply be added to the original image
to enhance edges. 

In [ ]:
def addLap(frac):
    global I, I_laplace
    temp = I + frac*I_laplace
    return np.clip(temp, 0,1)

I_halfL = addLap(.5)
I_1L = addLap(1)
I_2L = addLap(2)

In [ ]:
# Vis all three.
f, ax = plt.subplots(1,4, figsize=(10,3), sharex=True, sharey=True)
ax[0].imshow(I)
ax[0].set_title('Original')

ax[1].imshow(I_halfL)
ax[1].set_title('+.5*LaPlace')

ax[2].imshow(I_1L)
ax[2].set_title('+1*LaPlace')

ax[3].imshow(I_2L)
ax[3].set_title('+2*LaPlace')

plt.suptitle('Adding Edginess to Cat')

&nbsp;

## Interactive Visualization 

See [jupyter-matplotlib](https://github.com/matplotlib/jupyter-matplotlib/blob/master/examples/ipympl.ipynb)

In [ ]:
from ipywidgets import VBox, FloatSlider

plt.ioff()
plt.clf()

slider = FloatSlider(
    orientation='horizontal',
    value=1.0,
    min=0.00,
    max=5.0,
    description='How much Laplace to add'
)

fig_args = {'num':' ', 'frameon':True, 'sharex':True, 'sharey':True}
fig, ax = plt.subplots(1,2, figsize=(8,4), **fig_args) 

I_L = addLap(1);

# display artists I'll update
adisp = ax[0].imshow(I)
ldisp = ax[1].imshow(I_L)

ltext = ax[1].set_title('1*LaPlace')


def update_image(change):
    global I, I_laplace, adisp, ldisp, I_L, ltext
    I_L = addLap(change.new)
    ldisp.set_array(I_L)
    ltext.set_text('{}*LaPlace'.format(change.new))
    fig.canvas.draw()
    fig.canvas.flush_events()

slider.observe(update_image, names='value')

VBox([slider, fig.canvas])